# Workbench

In [36]:
from sklearn import datasets
from sklearn import preprocessing

X, y = datasets.fetch_openml('mnist_784', return_X_y=True)
X = preprocessing.scale(X)

/Users/max/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state=42)
X_fit, X_val, y_fit, y_val = model_selection.train_test_split(X_train, y_train, random_state=42)

In [3]:
from sklearn import ensemble
from sklearn import metrics

clf = ensemble.RandomForestClassifier(n_estimators=20, random_state=42)
clf.fit(X_fit, y_fit)
y_pred = clf.predict(X_val)
print(metrics.classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      1298
           1       0.97      0.98      0.98      1477
           2       0.94      0.96      0.95      1315
           3       0.94      0.93      0.93      1301
           4       0.95      0.95      0.95      1277
           5       0.94      0.93      0.94      1217
           6       0.97      0.97      0.97      1301
           7       0.96      0.96      0.96      1366
           8       0.94      0.93      0.94      1289
           9       0.95      0.94      0.94      1284

    accuracy                           0.95     13125
   macro avg       0.95      0.95      0.95     13125
weighted avg       0.95      0.95      0.95     13125



In [4]:
labels = clf.classes_.tolist()
confusion = metrics.confusion_matrix(y_val, y_pred, labels=clf.classes_)
pd.DataFrame(confusion, index=labels, columns=labels)

,0,1,2,3,4,5,6,7,8,9
0,1282,1,5,0,0,2,4,1,3,0
1,0,1454,8,3,2,2,1,3,1,3
2,4,2,1257,8,10,1,7,12,13,1
3,2,6,25,1205,1,27,1,8,20,6
4,2,4,4,0,1219,2,9,3,7,27
5,13,4,5,24,5,1135,8,4,14,5
6,17,5,1,0,5,15,1257,0,1,0
7,6,5,19,3,8,1,0,1310,2,12
8,3,8,8,25,7,17,5,5,1200,11
9,3,3,7,14,26,5,0,12,13,1201


In [40]:
import random
from myriad.myriad import make_balanced_tree

tree = make_balanced_tree(labels, shuffle=True)
tree

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 7.0.2 (20221119.0110)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 0 -->
 
 0 
 
 
<!-- 1 -->
 
 1 
 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 10 -->
 
 10 
 
 
<!-- 0--10 -->
 
 0--10 
 
 
<!-- 2 -->
 
 2 
 
 
<!-- 1--2 -->
 
 1--2 
 
 
<!-- 7 -->
 
 7 
 
 
<!-- 1--7 -->
 
 1--7 
 
 
<!-- 3 -->
 
 3 
 
 
<!-- 2--3 -->
 
 2--3 
 
 
<!-- 6 -->
 
 6 
 
 2 
 
<!-- 2--6 -->
 
 2--6 
 
 
<!-- 4 -->
 
 4 
 
 0 
 
<!-- 3--4 -->
 
 3--4 
 
 
<!-- 5 -->
 
 5 
 
 1 
 
<!-- 3--5 -->
 
 3--5 
 
 
<!-- 8 -->
 
 8 
 
 3 
 
<!-- 7--8 -->
 
 7--8 
 
 
<!-- 9 -->
 
 9 
 
 4 
 
<!-- 7--9 -->
 
 7--9 
 
 
<!-- 11 -->
 
 11 
 
 
<!-- 10--11 -->
 
 10--11 
 
 
<!-- 16 -->
 
 16 
 
 
<!-- 10--16 -->
 
 10--16 
 
 
<!-- 12 -->
 
 12 
 
 
<!-- 11--12 -->
 
 11--12 
 
 
<!-- 15 -->
 
 15 
 
 7 
 
<!-- 11--15 -->
 
 11--15 
 
 
<!-- 13 -->
 
 13 
 
 5 
 
<!-- 12--13 -->
 
 12--13 
 
 
<!-- 14 -->
 
 14 
 
 6 
 
<!-- 12--14 -->
 
 12--14 
 
 
<!-- 17 -->
 
 17 
 
 8 
 
<!-- 16--17 -->
 
 16--17 
 
 
<!-- 18 -->
 
 18 
 
 9 
 
<!-- 16--18 -->
 
 16--18

In [31]:
%run myriad/myriad.py

from sklearn import linear_model

clf = linear_model.LogisticRegression(random_state=42)

_ = train(tree, clf, X_fit, y_fit)
y_pred = predict(tree, X_val)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression',
                 LogisticRegression(max_iter=1000, random_state=42))])